# Initial Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv("./kieranFeatures_1-30_26-Sep-2024.csv")
df.head()

,ID,EDA_TonicMean_version02,EDA_TonicMean_version03,EDA_TonicMean_version04,EDA_TonicMean_version05,EDA_TonicMean_version09,EDA_TonicMean_version10,EDA_TonicMean_version11,EDA_TonicMean_version12,EDA_TonicMean_version16,...,EEG_avgRelTheta_version16,EEG_avgRelTheta_version17,EEG_avgRelTheta_version19,EEG_avgRelTheta_version20,EEG_avgRelTheta_version22,EEG_avgRelTheta_version23,adjSA1,adjSA2,adjSA3,adjSAtotal
0,5,-0.123031,-0.226077,-1.220480,-1.697738,-0.273200,-0.601171,-0.809518,-1.012558,-0.299118,...,-1.877017,-1.442056,1.070298,1.277417,0.249605,0.400156,0.119790,1.593122,-0.800726,0.350233
1,5,-0.152896,-0.050866,1.527067,1.883468,-0.378060,-0.018812,1.023216,1.189124,-0.355315,...,-1.632698,-1.531970,1.779032,1.074498,0.409991,0.333842,0.075246,-1.663383,0.859309,-0.262893
2,5,-0.166035,-0.181478,1.634437,0.904620,-0.424192,-0.452936,1.123414,0.534554,-0.380039,...,-1.489450,-1.448590,2.194570,1.262672,0.504028,0.395338,-1.072729,0.879836,-1.542415,-0.938513
3,5,-0.231095,-0.209571,1.654951,1.247081,-0.652624,-0.546311,1.214370,0.821624,-0.502463,...,-1.353433,-1.059878,2.589134,2.139926,0.593317,0.682023,-0.643181,-0.217332,0.945816,0.145041
4,5,-0.236090,-0.323013,-0.478244,-1.080788,-0.670161,-0.923364,-0.421866,-0.775114,-0.511862,...,-1.443846,-0.627980,2.326862,3.114644,0.533965,1.000560,-0.323098,0.712401,-1.473404,-0.642872


# Pre-Processing

In [ ]:
# Create binary variables for high and low 
adj_SA_1_median = np.median(df["adjSA1"])
adj_SA_2_median = np.median(df["adjSA2"])
adj_SA_3_median = np.median(df["adjSA3"])
adj_SA_tot_median = np.median(df["adjSAtotal"])

# Will be high if adjusted SA level score is equal to or above median, low otherwise
df["Lv_1_Hi"] = (df["adjSA1"] >= adj_SA_1_median).astype(int)
df["Lv_2_Hi"] = (df["adjSA2"] >= adj_SA_2_median).astype(int)
df["Lv_3_Hi"] = (df["adjSA3"] >= adj_SA_3_median).astype(int)
df["Tot_Hi"] = (df["adjSAtotal"] >= adj_SA_tot_median).astype(int)

Divide up dataframe into predictors and outcomes. Train-test-split the following data.

In [ ]:
predictors_df = df.iloc[:, 1:(df.shape[1] - 8)]
outcomes_df = df.iloc[:, (df.shape[1] - 8):]

display(predictors_df)
display(outcomes_df)

# Split into train and test
predictors_train, predictors_test, outcomes_train, outcomes_test = train_test_split(predictors_df, outcomes_df, test_size = 0.2, random_state = 42)

display(predictors_train)
display(outcomes_train)

# Scale the data
scaler = StandardScaler()
predictors_train = scaler.fit_transform(predictors_train)
predictors_train = pd.DataFrame(predictors_train, columns = predictors_df.columns)
predictors_test = scaler.transform(predictors_test)
predictors_test = pd.DataFrame(predictors_test, columns = predictors_df.columns)

display(predictors_train)
display(outcomes_train)

# Free up memory
del df
del predictors_df
del outcomes_df

,EDA_TonicMean_version02,EDA_TonicMean_version03,EDA_TonicMean_version04,EDA_TonicMean_version05,EDA_TonicMean_version09,EDA_TonicMean_version10,EDA_TonicMean_version11,EDA_TonicMean_version12,EDA_TonicMean_version16,EDA_TonicMean_version17,...,EEG_avgRelTheta_version09,EEG_avgRelTheta_version10,EEG_avgRelTheta_version11,EEG_avgRelTheta_version12,EEG_avgRelTheta_version16,EEG_avgRelTheta_version17,EEG_avgRelTheta_version19,EEG_avgRelTheta_version20,EEG_avgRelTheta_version22,EEG_avgRelTheta_version23
0,-0.123031,-0.226077,-1.220480,-1.697738,-0.273200,-0.601171,-0.809518,-1.012558,-0.299118,-0.469374,...,-2.470055,-1.633813,-1.521523,-1.189742,-1.877017,-1.442056,1.070298,1.277417,0.249605,0.400156
1,-0.152896,-0.050866,1.527067,1.883468,-0.378060,-0.018812,1.023216,1.189124,-0.355315,-0.160570,...,-1.999027,-1.796969,-0.890211,-0.846923,-1.632698,-1.531970,1.779032,1.074498,0.409991,0.333842
2,-0.166035,-0.181478,1.634437,0.904620,-0.424192,-0.452936,1.123414,0.534554,-0.380039,-0.390771,...,-1.722859,-1.645669,-0.543299,-0.588502,-1.489450,-1.448590,2.194570,1.262672,0.504028,0.395338
3,-0.231095,-0.209571,1.654951,1.247081,-0.652624,-0.546311,1.214370,0.821624,-0.502463,-0.440284,...,-1.460630,-0.940319,-0.955926,-0.744128,-1.353433,-1.059878,2.589134,2.139926,0.593317,0.682023
4,-0.236090,-0.323013,-0.478244,-1.080788,-0.670161,-0.923364,-0.421866,-0.775114,-0.511862,-0.640221,...,-1.634937,-0.156605,-0.344389,0.214848,-1.443846,-0.627980,2.326862,3.114644,0.533965,1.000560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,-0.390463,-0.392143,-0.150550,-0.112208,-0.248912,-0.241573,-0.239724,-0.191818,-0.336535,-0.330997,...,-0.046622,-0.464843,0.523703,0.110708,0.163210,-0.308442,0.105745,-0.391989,0.798338,-0.432294
300,-0.369596,-0.380586,-0.609280,-0.487820,-0.175647,-0.203160,-0.582746,-0.450147,-0.286580,-0.305082,...,-0.102853,-0.200243,0.539456,0.390624,0.101655,-0.000704,0.021136,-0.062902,0.571962,0.092100
301,-0.457362,-0.382835,-1.247644,-0.132967,-0.483803,-0.210636,-1.131586,-0.204537,-0.496694,-0.310125,...,0.160880,-0.095837,2.240786,1.697116,0.390354,0.120724,0.417964,0.066950,1.633695,0.299016
302,-0.370669,-0.390531,-1.078873,-0.866528,-0.179415,-0.236215,-0.917973,-0.699119,-0.289149,-0.327382,...,-0.394919,-0.514674,-0.292277,-0.368327,-0.218058,-0.366396,-0.418322,-0.453964,-0.603831,-0.531050


,adjSA1,adjSA2,adjSA3,adjSAtotal,Lv_1_Hi,Lv_2_Hi,Lv_3_Hi,Tot_Hi
0,0.119790,1.593122,-0.800726,0.350233,1,1,0,1
1,0.075246,-1.663383,0.859309,-0.262893,0,0,1,0
2,-1.072729,0.879836,-1.542415,-0.938513,0,1,0,0
3,-0.643181,-0.217332,0.945816,0.145041,0,0,1,0
4,-0.323098,0.712401,-1.473404,-0.642872,0,1,0,0
...,...,...,...,...,...,...,...,...
299,0.076099,1.105227,-0.609431,0.209332,0,1,0,1
300,-0.258249,-0.360422,0.778641,0.155357,0,0,1,1
301,0.110240,0.092504,0.945232,0.627581,1,1,1,1
302,-1.105639,0.426616,0.328063,-0.108335,0,1,1,0


,EDA_TonicMean_version02,EDA_TonicMean_version03,EDA_TonicMean_version04,EDA_TonicMean_version05,EDA_TonicMean_version09,EDA_TonicMean_version10,EDA_TonicMean_version11,EDA_TonicMean_version12,EDA_TonicMean_version16,EDA_TonicMean_version17,...,EEG_avgRelTheta_version09,EEG_avgRelTheta_version10,EEG_avgRelTheta_version11,EEG_avgRelTheta_version12,EEG_avgRelTheta_version16,EEG_avgRelTheta_version17,EEG_avgRelTheta_version19,EEG_avgRelTheta_version20,EEG_avgRelTheta_version22,EEG_avgRelTheta_version23
269,-0.494618,-0.555541,-2.076390,-1.869410,0.597167,0.362062,-1.765695,-1.479098,0.418093,0.161042,...,0.462172,0.859784,-0.051114,0.233826,0.237753,0.649436,0.111443,0.465456,0.779651,0.627392
211,-0.495506,-0.500435,-0.131699,-0.129467,-0.662698,-0.644065,-0.258565,-0.237457,-0.612173,-0.596055,...,-0.260123,1.010109,-0.213657,0.601153,-0.307295,0.907735,-0.047399,1.014504,0.237016,1.841947
197,0.297909,0.310984,-1.254804,-0.683124,-0.014281,0.030278,-0.612785,-0.323310,-0.147675,-0.124319,...,-1.458536,-2.291505,-0.823308,-1.187300,-1.355405,-1.868532,-0.703187,-1.310684,-0.785986,-1.462410
75,-0.751382,-0.767114,0.220138,-0.002063,-1.119621,-1.112651,0.082926,-0.217642,-1.034901,-1.023674,...,-0.084950,-0.236394,-1.212890,-1.107865,-0.042654,-0.178245,-0.830580,-0.678148,-1.206773,-0.626839
177,-0.354384,-0.332566,-1.195093,-0.569050,-1.246429,-1.107403,-0.981167,-0.478398,-0.795857,-0.715333,...,0.509455,0.654922,0.548907,0.656188,0.507635,0.712548,-0.440298,0.200804,-0.601341,0.526651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,-0.297054,-0.307219,-0.661230,-0.513394,-1.014946,-0.994582,-0.575531,-0.431628,-0.679828,-0.661295,...,-0.726769,-1.195044,-0.629046,-0.860808,-0.798998,-1.181357,0.012439,-0.598611,0.566104,-0.681974
71,-0.559570,-0.562361,1.381119,0.854606,-0.446144,-0.432101,1.443925,0.759285,-0.517737,-0.506605,...,0.597457,0.281063,-0.908693,-0.867127,0.619710,0.355382,0.196207,-0.034579,0.716476,0.070465
106,-0.362930,-0.376268,-0.392341,-0.370327,-0.500164,-0.518055,-0.417380,-0.369009,-0.471410,-0.478937,...,1.048729,1.497494,-0.488310,-0.037478,1.458071,2.088352,-0.736493,0.362864,-1.311561,0.741032
270,-0.663595,-0.680655,-0.323792,-0.360686,0.003870,-0.053789,-0.548041,-0.547836,-0.197162,-0.265638,...,0.846641,1.038962,0.487292,0.634474,0.559359,0.808679,0.689936,0.688303,2.265877,0.874194


,adjSA1,adjSA2,adjSA3,adjSAtotal,Lv_1_Hi,Lv_2_Hi,Lv_3_Hi,Tot_Hi
269,0.143670,1.101324,-0.822506,0.117187,1,1,0,0
211,-0.783965,-0.834554,0.116317,-0.677632,0,0,0,0
197,0.475912,-0.162414,1.180378,0.804260,1,0,1,1
75,-0.088596,0.229732,0.972713,0.618738,0,1,1,1
177,-1.146056,0.662741,-3.060939,-1.932524,0,1,0,0
...,...,...,...,...,...,...,...,...
188,1.116062,0.313836,0.270247,0.799881,1,1,1,1
71,-0.881938,0.311514,-0.739256,-0.666210,0,1,0,0
106,-0.900207,0.372292,-1.423042,-1.032613,0,1,0,0
270,-2.025591,0.753425,0.139548,-0.472791,0,1,0,0


,EDA_TonicMean_version02,EDA_TonicMean_version03,EDA_TonicMean_version04,EDA_TonicMean_version05,EDA_TonicMean_version09,EDA_TonicMean_version10,EDA_TonicMean_version11,EDA_TonicMean_version12,EDA_TonicMean_version16,EDA_TonicMean_version17,...,EEG_avgRelTheta_version09,EEG_avgRelTheta_version10,EEG_avgRelTheta_version11,EEG_avgRelTheta_version12,EEG_avgRelTheta_version16,EEG_avgRelTheta_version17,EEG_avgRelTheta_version19,EEG_avgRelTheta_version20,EEG_avgRelTheta_version22,EEG_avgRelTheta_version23
0,-0.498774,-0.560976,-2.050127,-1.854451,0.577210,0.339142,-1.697864,-1.405132,0.394787,0.143846,...,0.454675,0.827464,-0.030701,0.300150,0.224543,0.608457,0.108515,0.414417,0.830405,0.605829
1,-0.499679,-0.504911,-0.144123,-0.145626,-0.690567,-0.675169,-0.258050,-0.237600,-0.613827,-0.600710,...,-0.263765,0.979137,-0.216744,0.755464,-0.320508,0.866908,-0.045040,0.958251,0.276126,1.835104
2,0.308872,0.320641,-1.244885,-0.689381,-0.038077,0.004659,-0.596449,-0.318328,-0.159091,-0.136788,...,-1.455783,-2.352085,-0.914538,-1.461383,-1.368624,-1.910998,-0.678999,-1.344857,-0.768828,-1.509301
3,-0.760436,-0.776234,0.200714,-0.020500,-1.150359,-1.147566,0.068188,-0.218968,-1.027671,-1.021246,...,-0.089527,-0.278544,-1.360446,-1.362921,-0.055866,-0.219713,-0.802152,-0.718327,-1.198644,-0.663603
4,-0.355864,-0.334118,-1.186363,-0.577347,-1.277963,-1.142275,-0.948377,-0.464160,-0.793650,-0.718012,...,0.501706,0.620765,0.656070,0.823681,0.494427,0.671606,-0.424861,0.152278,-0.580220,0.503866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,-0.297441,-0.308329,-0.663120,-0.522686,-1.045026,-1.028537,-0.560859,-0.420182,-0.680061,-0.664869,...,-0.727921,-1.245791,-0.692190,-1.056685,-0.812214,-1.223418,0.012807,-0.639546,0.612276,-0.719407
239,-0.564964,-0.567915,1.338598,0.820848,-0.472653,-0.461481,1.368398,0.699650,-0.521375,-0.512742,...,0.589238,0.243553,-1.012268,-1.064518,0.606502,0.314229,0.190458,-0.080869,0.765875,0.042152
240,-0.364573,-0.378581,-0.399581,-0.382179,-0.527012,-0.548134,-0.409772,-0.361300,-0.476023,-0.485532,...,1.038101,1.470893,-0.531106,-0.036140,1.444868,2.048222,-0.711197,0.312800,-1.305680,0.720846
241,-0.670974,-0.688270,-0.332395,-0.372710,-0.019813,-0.080091,-0.534597,-0.529454,-0.207538,-0.275766,...,0.837092,1.008249,0.585547,0.796767,0.546151,0.767793,0.667753,0.635148,2.348523,0.855622


,adjSA1,adjSA2,adjSA3,adjSAtotal,Lv_1_Hi,Lv_2_Hi,Lv_3_Hi,Tot_Hi
269,0.143670,1.101324,-0.822506,0.117187,1,1,0,0
211,-0.783965,-0.834554,0.116317,-0.677632,0,0,0,0
197,0.475912,-0.162414,1.180378,0.804260,1,0,1,1
75,-0.088596,0.229732,0.972713,0.618738,0,1,1,1
177,-1.146056,0.662741,-3.060939,-1.932524,0,1,0,0
...,...,...,...,...,...,...,...,...
188,1.116062,0.313836,0.270247,0.799881,1,1,1,1
71,-0.881938,0.311514,-0.739256,-0.666210,0,1,0,0
106,-0.900207,0.372292,-1.423042,-1.032613,0,1,0,0
270,-2.025591,0.753425,0.139548,-0.472791,0,1,0,0


Use VIF to detect multicollinearity (Takes too long to run)

In [ ]:
# def calculate_VIF(df):
#     vif = pd.DataFrame()
#     vif["Features"] = df.columns
#     vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
#     return vif

# display(calculate_VIF(predictors_df))

/home/jshen/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss
